In [1]:
import pandas as pd

# add import statements
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from textblob import TextBlob
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from nltk.corpus import stopwords
from nltk.cluster import KMeansClusterer, \
cosine_distance
from sklearn import metrics
import numpy as np
from sklearn.cluster import KMeans
from sklearn import mixture
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation
import gensim
from gensim import corpora
import text2emotion as te
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report


[nltk_data] Downloading package stopwords to /Users/Kenny/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Kenny/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Kenny/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
movie_df = pd.read_csv("preprocessed_data.csv")
movie_df

,Unnamed: 0,userId,reviewDate,reviewScore,reviewTitle,userReview,movieId
0,0,2509775,26 November 2003,10,Tied for the best movie I have ever seen,Why do I want to write the 234th comment on Th...,111161
1,1,1898687,10 February 2006,10,A classic piece of unforgettable film-making.,"In its Oscar year, Shawshank Redemption (writt...",111161
2,2,16161013,24 July 2010,10,Some birds aren't meant to be caged.,The Shawshank Redemption is written and direct...,111161
3,3,1005460,8 February 2001,10,Prepare to be moved,I have never seen such an amazing film since I...,111161
4,4,997166,27 August 2002,10,Shawshank Redeems Hollywood,"Can Hollywood, usually creating things for ent...",111161
...,...,...,...,...,...,...,...
24183,24183,6729,17 June 2003,9,Changed my mind ...,This post refers to the extended DVD special e...,99348
24184,24184,2349843,9 May 2003,10,Read The Book,This is specifically for Tom Hull...did you ha...,99348
24185,24185,1945054,4 December 2002,10,One of the best films in the 90's,Dances with wolves is a very epic film.I must ...,99348
24186,24186,1651782,29 November 2002,10,Exquisite,They don't make 'em like this anymore. Marvelo...,99348


In [3]:
# Extract adjectives from the text
def get_adjectives(text):
    blob = TextBlob(text)
    return [ word for (word,tag) in blob.tags if tag == "JJ"]

movie_df['adjectives'] = movie_df['userReview'].apply(get_adjectives)

In [4]:
# if the reviews less than three words, drop this row to reduce the error
def merge_list(list_):
    return ' '.join(list_)

movie_df['adjectives_merge'] = movie_df['adjectives'].apply(merge_list)

movie_df['a_length'] = movie_df['adjectives'].apply(len)
movie_df = movie_df[movie_df['a_length'] >= 3]

In [5]:
movie_df

,Unnamed: 0,userId,reviewDate,reviewScore,reviewTitle,userReview,movieId,adjectives,adjectives_merge,a_length
0,0,2509775,26 November 2003,10,Tied for the best movie I have ever seen,Why do I want to write the 234th comment on Th...,111161,"[sure, many, other, simple, eloquent, only, ot...",sure many other simple eloquent only other fee...,16
1,1,1898687,10 February 2006,10,A classic piece of unforgettable film-making.,"In its Oscar year, Shawshank Redemption (writt...",111161,"[happy, good, all-time, huge, American, easy, ...",happy good all-time huge American easy mere ma...,61
2,2,16161013,24 July 2010,10,Some birds aren't meant to be caged.,The Shawshank Redemption is written and direct...,111161,"[tough, fellow, warden, much, unsure, several,...",tough fellow warden much unsure several mythic...,36
3,3,1005460,8 February 2001,10,Prepare to be moved,I have never seen such an amazing film since I...,111161,"[amazing, great, different, simple, everlastin...",amazing great different simple everlasting out...,10
4,4,997166,27 August 2002,10,Shawshank Redeems Hollywood,"Can Hollywood, usually creating things for ent...",111161,"[meticulous, due, Such, capable, undeniable, m...",meticulous due Such capable undeniable manager...,78
...,...,...,...,...,...,...,...,...,...,...
24183,24183,6729,17 June 2003,9,Changed my mind ...,This post refers to the extended DVD special e...,99348,"[special, multiple, rare, fine, original, thea...",special multiple rare fine original theatrical...,26
24184,24184,2349843,9 May 2003,10,Read The Book,This is specifically for Tom Hull...did you ha...,99348,"[very, same, other, western, outstanding, outs...",very same other western outstanding outstandin...,8
24185,24185,1945054,4 December 2002,10,One of the best films in the 90's,Dances with wolves is a very epic film.I must ...,99348,"[epic, long, little, boring, good, great, nati...",epic long little boring good great native fabu...,12
24186,24186,1651782,29 November 2002,10,Exquisite,They don't make 'em like this anymore. Marvelo...,99348,"[Marvelous, Great, entire, gorgeous, anti-Cost...",Marvelous Great entire gorgeous anti-Costner,5


In [6]:
movie_df.to_csv('data_adjs.csv')

In [7]:
# Train Test split.
train, test = train_test_split(movie_df['adjectives_merge'], test_size=0.1, random_state=42)

In [8]:
# Define the function get_emotion to get the text emtions
def get_emotion(sentence):
    e = te.get_emotion(sentence)
    return max(e, key = e.get)

In [9]:
train_df = train.to_frame()

In [10]:
train_df 

,adjectives_merge
16276,British in-between former tragic alcohol-drive...
1683,serial negative directorial dark clinically ex...
23048,hilarious graphic excessive hysterical sacrile...
5694,many excellent perfectionist same nervous impo...
12505,sure perfect fantastic perfect beautiful emoti...
...,...
12889,great many last only slight obvious different ...
22942,soft many amazing many phenomenal strong fit l...
5870,great historical second great wartime first we...
983,remarkable friendly big American congressional...


In [11]:
# Using get emotion package to get the sentiment label and add to test_df for eternal evaluation
test_df = test.to_frame()
test_df['label'] = test_df['adjectives_merge'].apply(get_emotion)

In [12]:
test_df

,adjectives_merge,label
21336,outstanding true digital investigative remarka...,Happy
12187,commented easy German 'evil Little little Germ...,Angry
12796,Real Real terrible sole simple Wild hot ex-bab...,Sad
14520,Sure outlandishly true adrenaline-pumping firs...,Happy
20162,main real real ethnic little common little oth...,Sad
...,...,...
2398,male abnormal amazing early adoring male human...,Happy
779,outstanding stunning fantastic,Happy
23733,original good same entertaining boring,Happy
301,perfect perfect perfect first much perfect wor...,Sad


In [19]:
def cluster_kmean(train, test, num_clusters = 2, distance ='cosine_distance'):
    
    predict = None
    
    drop_lists = ['much', 'first', 'many', 'non', 'human', 'last', 'main', 'sure', 'anti', 'top', 'whole', \
                 'little', 'able', 'true', 'hard', 'final', 'full', 'second', 'entire', \
                 'long', 'year', 'big', 'small', 'several', \
                 'short', 'overall', 'next', 'obvious', \
                 'self', 'right', 'actual', \
                 'third', 'personal']
    stop = list(stopwords.words('english')) + drop_lists
    
    tfidf_vect = TfidfVectorizer(stop_words=stop,\
                             min_df=3) 

    dtm= tfidf_vect.fit_transform(train)
    
    if distance == 'cosine_distance':
    
        clusterer = KMeansClusterer(num_clusters, \
                            cosine_distance, \
                            repeats=20)

        clusters = clusterer.cluster(dtm.toarray(), \
                             assign_clusters=True)     
        
        centroids=np.array(clusterer.means())

        sorted_centroids = centroids.argsort()[:, ::-1] 

        voc_lookup= tfidf_vect.get_feature_names()    

        for i in range(num_clusters):

            # get words with top 20 tf-idf weight in the centroid
            top_words=[voc_lookup[word_index] \
                       for word_index in sorted_centroids[i, :20]]
            print("Cluster %d:\n %s " % (i, "; ".join(top_words)))

        test_dtm = tfidf_vect.transform(test)

        predict = [clusterer.classify(v) for v in test_dtm.toarray()]

    elif distance == 'euclidean_distance':
        clusterer = KMeans(n_clusters=num_clusters, n_init=20, random_state = 42)\
            .fit(dtm)
        
        clusters = clusterer.labels_.tolist()
        
        centroids = clusterer.cluster_centers_

        sorted_centroids = centroids.argsort()[:, ::-1] 

        voc_lookup= tfidf_vect.get_feature_names()    

        for i in range(num_clusters):

            # get words with top 20 tf-idf weight in the centroid
            top_words=[voc_lookup[word_index] \
                       for word_index in sorted_centroids[i, :20]]
            print("Cluster %d:\n %s " % (i, "; ".join(top_words)))

        test_dtm = tfidf_vect.transform(test)
        
        predict = clusterer.predict(test_dtm)
        
    return predict

In [14]:
def external_evaluate(pred_clusters, test_labels):

    confusion_df = pd.DataFrame(list(zip(test_labels.values, pred_clusters)),\
                            columns = ["label", "cluster"])
    
    ct = pd.crosstab(index=confusion_df.cluster, columns=confusion_df.label)
    ct_max = ct.idxmax(axis = 1)
    cluster_dict={i:ct_max.iloc[i] for i in range(5)}

    predicted_target=[cluster_dict[i] \
                      for i in pred_clusters]
    print(metrics.classification_report\
      (test_labels, predicted_target))

In [20]:
#Kmeans clustering results
pred_clusters_kmean = cluster_kmean(train_df['adjectives_merge'], test_df['adjectives_merge'], num_clusters=5, distance='cosine_distance')

/opt/anaconda3/lib/python3.8/site-packages/nltk/cluster/util.py:131: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (numpy.dot(u, v) / (sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))


Cluster 0:
 amazing; perfect; incredible; favorite; excellent; brilliant; fantastic; wonderful; great; good; beautiful; classic; awesome; special; different; favourite; funny; emotional; memorable; superb 
Cluster 1:
 great; good; wonderful; classic; funny; old; excellent; perfect; fantastic; different; brilliant; favorite; interesting; beautiful; special; new; original; bad; young; important 
Cluster 2:
 good; bad; great; boring; interesting; funny; old; high; excellent; wrong; classic; different; original; new; special; nice; stupid; brilliant; real; slow 
Cluster 3:
 real; great; good; old; young; different; bad; beautiful; new; american; excellent; brilliant; perfect; important; special; simple; realistic; wonderful; interesting; funny 
Cluster 4:
 old; young; new; different; beautiful; original; classic; black; powerful; white; emotional; american; important; simple; funny; high; special; interesting; great; wonderful 


In [21]:
# Kmeans external_evaluation
external_evaluate(pred_clusters_kmean, test_df['label'])

              precision    recall  f1-score   support

       Angry       0.00      0.00      0.00        39
        Fear       0.00      0.00      0.00       402
       Happy       0.57      0.94      0.71      1252
         Sad       0.38      0.26      0.31       321
    Surprise       0.00      0.00      0.00       261

    accuracy                           0.55      2275
   macro avg       0.19      0.24      0.20      2275
weighted avg       0.37      0.55      0.43      2275



/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
def cluster_lda(train, test_text, num_clusters=2):
    
    predict = None
    
    # add  your code
    # LDA can only use raw term counts for LDA 
    drop_lists = ['much', 'first', 'many', 'non', 'human', 'last', 'main', 'sure', 'anti', 'top', 'whole', \
                 'little', 'able', 'true', 'hard', 'final', 'full', 'second', 'entire', \
                 'long', 'year', 'big', 'small', 'several', \
                 'short', 'overall', 'next', 'obvious', \
                 'self', 'right', 'actual', \
                 'third', 'personal']
    stop = list(stopwords.words('english')) + drop_lists
    
    tf_vectorizer = CountVectorizer(stop_words=stop,\
                             min_df=3)
    
    tf = tf_vectorizer.fit_transform(train)
    
    tf_feature_names = tf_vectorizer.get_feature_names()

    num_topics = num_clusters


    lda = LatentDirichletAllocation(n_components=num_topics, \
                                    max_iter=30,verbose=1,
                                    evaluate_every=1, n_jobs=-1,
                                    random_state=0).fit(tf)
    
    num_top_words=20
    
    for topic_idx, topic in enumerate(lda.components_):
        print ("Topic %d:" % (topic_idx))
        # print out top 20 words per topic 
        words=[(tf_feature_names[i],'%.2f'%topic[i]) \
               for i in topic.argsort()[::-1][0:num_top_words]]
        print(words)
        print("\n")

    test_dtm = tf_vectorizer.transform(test_text)

    lda_test = lda.transform(test_dtm.toarray())
    
    predict = pd.DataFrame(lda_test).idxmax(axis = 1).tolist()
    
    return predict
        

In [17]:
# LDA clustering results
pred_clusters_lda = cluster_lda(train_df['adjectives_merge'], test_df['adjectives_merge'], num_clusters=5)

iteration: 1 of max_iter: 30, perplexity: 1675.1435
iteration: 2 of max_iter: 30, perplexity: 1630.6157
iteration: 3 of max_iter: 30, perplexity: 1603.0338
iteration: 4 of max_iter: 30, perplexity: 1582.3921
iteration: 5 of max_iter: 30, perplexity: 1565.6308
iteration: 6 of max_iter: 30, perplexity: 1552.0638
iteration: 7 of max_iter: 30, perplexity: 1541.2786
iteration: 8 of max_iter: 30, perplexity: 1532.5542
iteration: 9 of max_iter: 30, perplexity: 1525.4775
iteration: 10 of max_iter: 30, perplexity: 1519.8620
iteration: 11 of max_iter: 30, perplexity: 1515.1126
iteration: 12 of max_iter: 30, perplexity: 1511.2271
iteration: 13 of max_iter: 30, perplexity: 1507.8874
iteration: 14 of max_iter: 30, perplexity: 1505.0315
iteration: 15 of max_iter: 30, perplexity: 1502.3972
iteration: 16 of max_iter: 30, perplexity: 1500.1213
iteration: 17 of max_iter: 30, perplexity: 1498.0960
iteration: 18 of max_iter: 30, perplexity: 1496.3081
iteration: 19 of max_iter: 30, perplexity: 1494.6778
it

In [22]:
# LDA external_evaluation
external_evaluate(pred_clusters_lda, test_df['label'])

              precision    recall  f1-score   support

       Angry       0.00      0.00      0.00        39
        Fear       0.00      0.00      0.00       402
       Happy       0.55      1.00      0.71      1252
         Sad       0.00      0.00      0.00       321
    Surprise       0.00      0.00      0.00       261

    accuracy                           0.55      2275
   macro avg       0.11      0.20      0.14      2275
weighted avg       0.30      0.55      0.39      2275



/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
# manual drop lists

drop_lists1 = ['much', 'first', 'many', 'non', 'human', 'last', 'main', 'sure', 'anti', 'top', 'whole']
drop_lists2 = ['little', 'able', 'true', 'hard', 'final', 'full', 'second', 'entire']
drop_lists3 = ['long', 'year', 'big', 'small', 'several']
drop_lists4 = ['short', 'overall', 'next', 'obvious']
drop_lists5 = ['self', 'right', 'actual']
drop_lists6 = ['third', 'personal']